# Functions included in project1

In [ ]:
# remove stopwords from texts
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

In [ ]:
# generate the dominant topic and its percentage
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    sent_topics_df = pd.DataFrame()
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [ ]:
# get the word frequency in a specific school
def findcount(word,sch,ds):
    ds = ds.loc[ds.school==sch]
    sum = 0
    for texts in ds.tokenized_txt:
        sum += eval(texts).count(word)
    return(sum/len(ds))

In [ ]:
# return a dataframe with the top 2 word frequency in before and top 3 word frequency in after
def topschools(word):
    countsbef = []
    for sch in schoolsbef:
        countsbef.append(findcount(word,sch,bef))
    ds1 = pd.DataFrame({'counts': countsbef, 'sch': schoolsbef})
    ds1 = ds1.sort_values('counts')[1:]
    countsaft = []
    for sch in schoolsaft:
        countsaft.append(findcount(word,sch,aft))
    ds2 = pd.DataFrame({'counts': countsaft, 'sch': schoolsaft})
    ds2 = ds2.sort_values('counts')[7:]
    index = ['bef','aft']
    cols = ds1.sch.tolist()+ds2.sch.tolist()
    theds = pd.DataFrame(0,columns=cols,index=index)
    for col in ds1.sch:
        theds.loc['bef',col] = float(ds1.loc[ds1.sch==col,'counts'])
    for col in ds2.sch:
        theds.loc['aft',col] = float(ds2.loc[ds2.sch==col,'counts'])
    return(theds)

In [ ]:
# generate a list of bools of if the word is in the sentence
def iswoman(ds):
    isin = []
    for sentence in ds.sentence_lowered:
        if re.search(r'\bwoman\b',sentence):
            isin.append(True)
        else:
            isin.append(False)
    return isin
def isman(ds):
    isin = []
    for sentence in ds.sentence_lowered:
        if re.search(r'\bman\b',sentence):
            isin.append(True)
        else:
            isin.append(False)
    return isin
def isfe(ds):
    isin = []
    for sentence in ds.sentence_lowered:
        if re.search(r'\bfemale\b',sentence):
            isin.append(True)
        else:
            isin.append(False)
    return isin
def isma(ds):
    isin = []
    for sentence in ds.sentence_lowered:
        if re.search(r'\bmale\b',sentence):
            isin.append(True)
        else:
            isin.append(False)
    return isin

In [ ]:
# generate n most common words and their counts in sentences that mention the given word
def wordcount(ds,word,n,stop=stop_word):
    if word=='man':
        ds = ds.loc[isman(ds)]
    elif word=='woman':
        ds = ds.loc[iswoman(ds)]
    elif word=='female':
        ds = ds.loc[isfe(ds)]
    else:
        ds = ds.loc[isma(ds)]
    fetokens = [eval(tokens) for tokens in ds['tokenized_txt']]
    fetoken = remove_stopwords2(fetokens, stop_word)
    fetoken = [t for tokens in fetoken for t in tokens]
    counts_fe = Counter(fetoken)
    return(counts_fe.most_common(n))

In [ ]:
# judge if a sentence is positive
def sentiment_scores(sentence):
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(sentence)
    if sentiment_dict['compound'] >= 0.05 :
        return("Positive")
 
    elif sentiment_dict['compound'] <= - 0.05 :
        return("Negative")
 
    else :
        return("Neutral")


In [ ]:
# generate a pie chart including the positive, negative and neutral proportion in a given dataset
def senti(df):      
    corpus = ''
    numPostives = 0
    numNegatives = 0
    numNeutrals = 0
    
    for mem in df['sentence_lowered']:
        corpus += mem
    
    for i in range (len(df)):
        sent = (sentiment_scores(df['sentence_lowered'].iloc[i]))
        if sent == "Positive":
            numPostives += 1
        elif sent == "Negative":
            numNegatives += 1
        else:
            numNeutrals += 1
    
    if len(df)<100000:
        thetime = 'early'
    else:
        thetime = 'modern'

    plt.figure(figsize = (7, 7))
    plt.pie([numPostives, numNegatives, numNeutrals], labels = ['positives', 'negatives', 'neutrals'], autopct='%1.2f%%', colors = ['#ff9999','#66b3ff','#ffcc99'])
    plt.title('Sentiment Analysis for '+thetime+ ' Philosopher')